In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 41.2 MB/s 
     |████████████████████████████████| 182 kB 47.4 MB/s 


In [2]:
import transformers

In [5]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [6]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [7]:
tokenizer.tokenize("오늘은 날씨가 참 좋네요")

['오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요']

근데 혹시 고유명사 같은 경우에는 어떻게 처리하나?

1. 어떤 기법을 사용할지를 고려
2. vocab_size를 결정 >> 작으면 메모리 이득, 크면 의미를 더 잘 표현
3. 상황에 맞는 corpus를 결정 >> 통계적으로 분석, vocab을 결정
자주 사용되는 고유명사라면 이 과정에서 자연히 corpus에 추가되어 있을 것

혹시 그렇지 않더라도, tokenizer에 vocab을 수동으로 직접 추가할 수 있다!

vocab 300 ~ 350을 [UNK1], [UNK2], [UNK3]... 등으로 해서 여분의 자리로 남겨뒀다가, 필요시에 원하는 vocab으로 변경하는 방식도 있다

In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
tokenizer.tokenize('안녕하세요')

['ᄋ',
 '##ᅡ',
 '##ᆫ',
 '##ᄂ',
 '##ᅧ',
 '##ᆼ',
 '##ᄒ',
 '##ᅡ',
 '##ᄉ',
 '##ᅦ',
 '##ᄋ',
 '##ᅭ']

In [16]:
tokenizer.vocab_size

30522

In [13]:
multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [14]:
multi_tokenizer.tokenize('안녕하세요')

['안', '##녕', '##하', '##세', '##요']

In [15]:
multi_tokenizer.vocab_size

119547

In [17]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [20]:
tokenizer.tokenize('okay')

['ok', 'ay']

**3가지 종류 언어모델**

-> 언어에 대한 통계적 이해를 가진 모델

1. Decoder-only 모델 Auto regressive model(GPT1, GPT2, GPT3, ...)
  - few shot learning 사용
  - 모델이 너무 크다는 단점
  - 점점 더 모델이 커지고 있음(모델이 크고, 데이터를 많이 넣을수록 성능이 좋아진다.)


2. Encoder-only 모델 (BERT, RoBERTa, Albert, ...)
  - input을 vector로 바꿔준다.
  - vector는 의미와 관련이 있다

  - Pre-training(사전학습, 비지도 학습) >> fine-tuning(지도 학습)
  - Downstream task (번역, 문장 분류, 질의응답 등...)
  - Why?
    1. 성능이 기존 방식보다 훨씬 좋다
    2. 소량의 데이터로도 학습이 훨씬 빠르게 된다

3. Encoder-Decoder 모델 (BART, T5, 등) >> Transformer 모델과 구조는 동일하다

  - Pre-training(사전학습, 비지도 학습) >> fine-tuning(지도학습)

embeding??
ex)
고양이, 낙타, 의자 가 있을때
1. 고양이와 낙타 간의 유사도와
2. 낙타와 의자 간의 유사도를 비교하면?
 사람은 1이 더 유사하다고 생각
 근데 컴퓨터가 이걸 이해하려면?

각 단어들을 토큰화 하여 숫자로 바꾼 뒤, 이것을 다시 벡터로 변환!
이때, 이 벡터를 유사한 단어들 끼리 비슷하게 변환하면 컴퓨터도 유사도를 이해할 수 있을 것
이처럼 단어를 벡터로 바꾸는 과정, 레이어를 임베딩 이라고 한다

embeding? -> 훈련 가능
encoding? -> 훈련 불가

Encoder 모델은 불가하지만, Decoder 모델은 가능한 Task가 있다면?

-> '생성'

새로운 데이터를 생성할 수가 없다...!

따라서 Encoder는 추출은 가능하나 생성이 불가

만약, 우리가 챗봇을 만들고자 한다면?

시나리오  1. Encoder 모델로 챗봇 만들기?

-> 미리 준비된 답변들을 준비해두고, 그 중에서 질문에 대한 모범 답안을 찾아내도록 해야함!

시나리오 2. Decoder 모델을 사용할 경우?

-> 좀 더 사람같은 느낌을 줄 수 있지만, 어떤 대답을 할지 예측할 수 없다.

-> 문제소지가 있는 답변을 할 수도 있으므로 상대적으로 위험! 다른 대책이 필요하다.

Attention Mechanism?

특정 단어에 가중치를 줘서 예측과 관련있는 특정 부분에 좀 더 집중할 수 있도록 하는 메커니즘!

Q(Query), K(Keys), V(Values)의 세가지 벡터를 이용해서 Q와 K 값을 내적하여 V에 곱하는 식으로 특정 부분에 가중치를 줄 수 있도록 설계되었다.

Decoder에서 추론 시점에 Encoder 토큰을 얼마나 참조하는가?
-> Decoder에서 추론할 때에 Encoder를 참조하는 정도에 따라 가중치가 부여된다


Self-attention??


질의응답

[CLS] 포메라니안은 어떤걸 좋아해? [SEP] 포메라니안은 독일 태생의 사냥견으로, 노는걸 좋아한다. [SEP]

-> 노는걸

요즘 유행하는 달리 같은 text2image 모델은 어떻게 작동할까??

BERT >> 문장을 벡터로 만들어준다 >> 벡터를 이미지로 만든다